In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def choose_initial_cluster_centers(points, clusters):
    us = np.zeros((clusters,784))         # centers of each cluster
    indicies = np.zeros(clusters)

    index = np.copy(np.random.choice(points.shape[0], 1)[0])         # choose first point randomly
    indicies[0] = np.copy(index)
    us[0] =  np.copy(points[index])

    for i in range(1,clusters):
        max_distance = 0
        for j in range(points.shape[0]):    # choose the center with the biggest distancce form last center
            if j not in indicies:           # check if the the point is already a center to cluster
                point = np.copy(points[j])
                last_point = np.copy(us[i-1])
                distance = np.linalg.norm(point-last_point)     # Eucledian distance

                if distance > max_distance:
                    max_distance = distance
                    index = j
                    u = np.copy(point)
        
        
        indicies[i] = np.copy(index)
        us[i] = np.copy(u)

    return us

In [3]:
def compute_membership(centers, point, rnks):
    min_distance = np.linalg.norm(point-centers[0])

    index = 0
    for i in range(1,centers.shape[0]):
        
        center = np.copy(centers[i])
        distance = np.linalg.norm(point-center)

        if distance <= min_distance and distance > 0:
            min_distance = distance
            index = i
        # elif distance == min_distance and distance > 0:      # if the distances are the same between two centers, choose the one with more members
        #     if rnks[i] > rnks[index]:
        #         index = i

    return index, min_distance

In [10]:
def kmeans(points, clusters, labels):

    old_centers = np.zeros((clusters,784))
    centers = np.copy(choose_initial_cluster_centers(points, clusters))
    
    for limit in range(200):
        
        if (np.array_equal(centers, old_centers)):      # stop if the algorithm converges
            print("Iterations till convergence: ", limit)
            break

        # print("Iteration Number : ")
        # print(limit)

        points_sum = np.zeros((10,784))
        rnks = np.zeros(clusters)       # used for counting number of members in each cluster
        distances = np.zeros(clusters)  # sum of distances in each cluster from the points to the cluster centers
        old_centers = np.copy(centers)
        cluster_labels_count = np.zeros((10,10))

        for i in range(points.shape[0]):
            point = np.copy(points[i])
            rnk, distance = compute_membership(centers, point, rnks)    # returns the cluster number of the point and its distance from the mean of the cluster
            rnks[rnk] += 1
            distances[rnk] += distance
            points_sum[rnk] += point
            cluster_labels_count[rnk][int(labels[i])] += 1
        
        for i in range(clusters):
            if(rnks[i] != 0):
                centers[i] = np.copy(points_sum[i] / rnks[i])         # Adjust the cluster centers

    sum_distances = np.sum(distances)
    
    return centers, sum_distances, rnks, cluster_labels_count

In [5]:
def np_thresholding(npArray, threshold):
    to_zero = npArray < threshold
    to_ones = npArray >= threshold

    npArray[to_zero] = 0
    npArray[to_ones] = 1

In [6]:
images = []

for i in range(2400):
    img = plt.imread("Images/" + str((i+1)) + ".jpg")
    pixels = []
    for j in range(28):
        for k in range(28):
            pixels.append(img[j,k])
    
    images.append(pixels)
    
images = np.asarray(images)
np_thresholding(images, 140)
print(images.shape)

(2400, 784)


In [7]:
imagesT = np.transpose(images)
print(imagesT.shape)

(784, 2400)


In [8]:
labels = np.genfromtxt('Images/Training Labels.txt')
print(labels)

[0. 0. 0. ... 9. 9. 9.]


In [11]:
centers_arr = np.zeros((30,10,784))
distances_arr = np.zeros(30)
rnks = np.zeros((30,10))
cluster_labels_count = np.zeros((30,10,10))

for i in range(30):
    print("Iteration number: ", i)
    centers_arr[i], distances_arr[i], rnks[i], cluster_labels_count[i] = kmeans(images, 10, labels)

index_best = int(np.where(distances_arr == np.amin(distances_arr))[0])

centers_arr_best = centers_arr[index_best]
distances_arr_best = distances_arr[index_best]
rnks_best = rnks[index_best]
cluster_labels_count_best = cluster_labels_count[index_best]

print("Best iteration: ", index_best)
print(cluster_labels_count_best)

Iteration number:  0
Iteration number:  1
Iteration number:  2


In [0]:
cluster_labels_count_plot = np.copy(cluster_labels_count_best)

x = np.arange(10)
y = np.zeros(10)

for i in range(10):
    y[i] = np.amax(cluster_labels_count_plot[0:, i:(i+1)])
    # cluster_labels_count_plot[int(np.where(cluster_labels_count_plot == np.amax(cluster_labels_count_plot[0:, i : (i+1)]))[0][0])] *= 0

print(y)

# plt.figure()

plt.bar(x, y)
plt.title('K-means')
plt.savefig('Counts.jpg')
plt.show()